In [93]:
import requests
import pandas as pd
import numpy
import datetime

mazání duplikátů

In [94]:
r = requests.get("https://raw.githubusercontent.com/pesikj/progr2-python/master/data/data_with_ids.csv")
open("data_with_ids.csv", 'wb').write(r.content)

data = pd.read_csv ("data_with_ids.csv")
#
data


,bank_id,date,amount,recipient,sender
0,10624661858,2019-05-02,-3361.88,Salazar Ltd,NaN
1,10624662480,2019-05-06,-1492.87,Villanueva PLC,NaN
2,10624665475,2019-05-07,-8092.58,"Myers, Thornton and Hill",NaN
3,10624669251,2019-05-09,1346.77,NaN,Bell-Roberts
4,10624672123,2019-05-11,-4183.59,Levy Group,NaN
...,...,...,...,...,...
1216,10629615961,2026-02-28,-6537.46,Williams PLC,NaN
1217,10629621592,2026-03-01,-6223.10,Matthews-Rogers,NaN
1218,10629621592,2026-03-01,-6223.10,Matthews-Rogers,NaN
1219,10629626972,2026-03-01,9809.10,NaN,Wells-Randall


In [95]:
data.shape[0]
#počet záznamů


1221

In [96]:
data["bank_id"].nunique()
#počet unikátních bank_id

999

In [97]:
data_unique = data.drop_duplicates(ignore_index=True)
data_unique.shape[0]

999

In [98]:
r = requests.get("https://raw.githubusercontent.com/pesikj/progr2-python/master/data/covid_data.csv")
open("covid_data.csv", 'wb').write(r.content)

data_covid = pd.read_csv("covid_data.csv")
data_covid.head()

,Unnamed: 0,date,value
0,0,2021-07-01,42
1,1,2021-07-01,179
2,2,2021-07-02,249
3,3,2021-07-03,44
4,4,2021-07-03,64


In [99]:
data_covid = data_covid.drop_duplicates(subset=["date"], keep="last")
data_covid.head()

,Unnamed: 0,date,value
1,1,2021-07-01,179
2,2,2021-07-02,249
4,4,2021-07-03,64
5,5,2021-07-04,61
6,6,2021-07-05,184


práce s datem a časem

In [100]:
r = requests.get("https://raw.githubusercontent.com/pesikj/progr2-python/master/data/invoices.csv")
open("invoices.csv", 'wb').write(r.content)
data_invoices = pd.read_csv("invoices.csv")
data_invoices

OSError: [Errno 22] Invalid argument: 'invoices.csv'

In [ ]:
data_invoices.dtypes

invoice_date     object
customer         object
amount          float64
dtype: object

In [ ]:
data_invoices["datum_conv"] = pd.to_datetime(data_invoices["invoice_date"], format="%d. %m. %Y")
data_invoices.head()

,invoice_date,customer,amount,datum_conv
0,24. 07. 2021,Robbins-Bryant,42424.49,2021-07-24
1,03. 06. 2021,"Little, Espinoza and Manning",6269.53,2021-06-03
2,24. 06. 2021,Hartman-Garcia,19182.55,2021-06-24
3,09. 08. 2021,"Mason, Armstrong and Whitaker",66295.23,2021-08-09
4,29. 06. 2021,Dickerson-Blankenship,88592.58,2021-06-29


In [ ]:
data_invoices["due_date"] = data_invoices["datum_conv"] + pd.Timedelta("60 days")
# nebo podle iso normy ("P60D")
data_invoices.head()

,invoice_date,customer,amount,datum_conv,due_date
0,24. 07. 2021,Robbins-Bryant,42424.49,2021-07-24,2021-09-22
1,03. 06. 2021,"Little, Espinoza and Manning",6269.53,2021-06-03,2021-08-02
2,24. 06. 2021,Hartman-Garcia,19182.55,2021-06-24,2021-08-23
3,09. 08. 2021,"Mason, Armstrong and Whitaker",66295.23,2021-08-09,2021-10-08
4,29. 06. 2021,Dickerson-Blankenship,88592.58,2021-06-29,2021-08-28


In [ ]:
#porovnání splatnosti faktury
today_date = datetime.datetime.now()
data_invoices["status"] = numpy.where(data_invoices["due_date"] < today_date, "overdue", "OK")
data_invoices

,invoice_date,customer,amount,datum_conv,due_date,status
0,24. 07. 2021,Robbins-Bryant,42424.49,2021-07-24,2021-09-22,overdue
1,03. 06. 2021,"Little, Espinoza and Manning",6269.53,2021-06-03,2021-08-02,overdue
2,24. 06. 2021,Hartman-Garcia,19182.55,2021-06-24,2021-08-23,overdue
3,09. 08. 2021,"Mason, Armstrong and Whitaker",66295.23,2021-08-09,2021-10-08,OK
4,29. 06. 2021,Dickerson-Blankenship,88592.58,2021-06-29,2021-08-28,overdue
...,...,...,...,...,...,...
99995,22. 08. 2021,Jones Group,64350.38,2021-08-22,2021-10-21,OK
99996,02. 08. 2021,Cardenas-Ryan,62539.74,2021-08-02,2021-10-01,OK
99997,14. 06. 2021,Wagner-Cunningham,31354.43,2021-06-14,2021-08-13,overdue
99998,19. 06. 2021,"Lambert, Lewis and Thomas",11701.15,2021-06-19,2021-08-18,overdue


In [ ]:
data_invoices.groupby("status")["amount"].sum()

status
OK         1.272180e+09
overdue    3.715678e+09
Name: amount, dtype: float64

In [ ]:
r = requests.get("https://raw.githubusercontent.com/pesikj/progr2-python/master/data/invoices_2.csv")
open("invoices_2.csv", 'wb').write(r.content)

invoices2 = pd.read_csv("invoices_2.csv")
invoices2["invoice_date"] = pd.to_datetime(invoices2["invoice_date"], format="%d. %m. %Y")
invoices2["payment_date"] = pd.to_datetime(invoices2["payment_date"], format="%d. %m. %Y")
invoices2.head()

,invoice_date,customer,amount,payment_date
0,2016-05-01,Campos PLC,29366,2016-06-25
1,2016-05-10,"Garcia, Hamilton and Carr",61145,2016-07-16
2,2016-05-20,"Garcia, Hamilton and Carr",92659,2016-07-26
3,2016-05-25,"Nguyen, Harrell and Lee",40076,2016-08-05
4,2016-06-04,"Garcia, Hamilton and Carr",54854,2016-08-06


In [ ]:
#odstranění nezaplacených - nemají vyplněné payment date
invoices2_paid = invoices2.dropna().reset_index(drop=True)
invoices2_paid["paid_in"] = invoices2["payment_date"] - invoices2["invoice_date"]
invoices2_paid

,invoice_date,customer,amount,payment_date,paid_in
0,2016-05-01,Campos PLC,29366,2016-06-25,55 days
1,2016-05-10,"Garcia, Hamilton and Carr",61145,2016-07-16,67 days
2,2016-05-20,"Garcia, Hamilton and Carr",92659,2016-07-26,67 days
3,2016-05-25,"Nguyen, Harrell and Lee",40076,2016-08-05,72 days
4,2016-06-04,"Garcia, Hamilton and Carr",54854,2016-08-06,63 days
...,...,...,...,...,...
248,2021-05-28,"Summers, Barnett and Smith",81810,2021-07-26,59 days
249,2021-06-03,Wagner LLC,52467,2021-08-02,60 days
250,2021-06-13,"Nguyen, Harrell and Lee",44942,2021-08-20,68 days
251,2021-06-18,Grimes-Green,81156,2021-08-14,57 days


In [101]:
average_payment_data = invoices2_paid.groupby(["customer"])["paid_in"].mean(numeric_only=False)

average_payment_data


#average_payment_data = pd.DataFrame(average_payment_data)

customer
Archer-Patel                   46 days 20:34:17.142857143
Campos PLC                               60 days 00:00:00
Chang-Fisher                             63 days 05:20:00
Clayton-Cook                   69 days 22:09:13.846153846
Faulkner-Howard                74 days 06:32:43.636363636
Garcia, Hamilton and Carr                63 days 22:40:00
Grimes-Green                             52 days 04:00:00
Jones, Jefferson and Rivera              71 days 15:00:00
Levy Group                               65 days 00:00:00
Moon, Davis and Larsen         53 days 14:46:09.230769231
Nguyen, Harrell and Lee        67 days 05:08:34.285714286
Patrick, Barrera and Collins             52 days 09:36:00
Sheppard-Tucker                          61 days 02:24:00
Summers, Barnett and Smith               59 days 16:00:00
Wagner LLC                     58 days 05:08:34.285714286
Name: paid_in, dtype: timedelta64[ns]

In [ ]:
invoices2_not_paid = invoices2[invoices2["payment_date"].isna()]
invoices2_not_paid = pd.merge(invoices2_not_paid, average_payment_data, on=["customer"], how="outer")
invoices2_not_paid["expected_payment_date"] = invoices2_not_paid["invoice_date"] + invoices2_not_paid["paid_in"]
invoices2_not_paid["expected_payment_date"] = invoices2_not_paid["expected_payment_date"].dt.date
print(invoices2_not_paid.head())

  invoice_date                     customer   amount payment_date  \
0   2021-07-02      Nguyen, Harrell and Lee  93301.0          NaT   
1   2021-07-09                   Levy Group  43621.0          NaT   
2   2021-08-05                   Levy Group  39651.0          NaT   
3   2021-07-16                   Wagner LLC  65064.0          NaT   
4   2021-07-22  Jones, Jefferson and Rivera  98317.0          NaT   

                     paid_in expected_payment_date  
0 67 days 05:08:34.285714286            2021-09-07  
1           65 days 00:00:00            2021-09-12  
2           65 days 00:00:00            2021-10-09  
3 58 days 05:08:34.285714286            2021-09-12  
4           71 days 15:00:00            2021-10-01  


In [102]:
r = requests.get("https://raw.githubusercontent.com/pesikj/progr2-python/master/data/ioef.csv")
open("ioef.csv", 'wb').write(r.content)

ioef = pd.read_csv("ioef.csv")
ioef


,Name,Index Year,Overall Score,Property Rights,Government Integrity,Judicial Effectiveness,Tax Burden,Government Spending,Fiscal Health,Business Freedom,Labor Freedom,Monetary Freedom,Trade Freedom,Investment Freedom,Financial Freedom
0,Albania,2021,65.2,46.1,40.6,22.8,89.0,74.6,86.6,66.1,51.6,82.0,82.8,70.0,70.0
1,Armenia,2021,71.9,57.3,45.0,55.3,87.1,81.3,84.3,81.9,74.5,76.9,73.8,75.0,70.0
2,Austria,2021,73.9,86.8,84.8,83.5,45.7,29.1,90.0,72.6,68.4,81.7,84.0,90.0,70.0
3,Azerbaijan,2021,70.1,67.9,46.8,55.8,88.1,65.2,99.4,80.5,65.9,73.2,68.0,70.0,60.0
4,Belarus,2021,61.0,58.6,39.8,40.4,92.9,55.5,96.5,75.9,75.7,70.2,76.0,30.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,Sweden,1995,61.4,70.0,90.0,NaN,44.0,0.0,NaN,70.0,NaN,81.5,77.0,70.0,50.0
1221,Switzerland,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1222,Turkey,1995,58.4,70.0,10.0,NaN,44.5,63.3,NaN,85.0,NaN,38.2,74.8,70.0,70.0
1223,Ukraine,1995,39.9,30.0,10.0,NaN,61.8,47.1,NaN,55.0,NaN,0.0,55.0,50.0,50.0


In [103]:
ioef["Rank"] = ioef.groupby(["Index Year"])["Overall Score"].rank(ascending=False)
ioef

,Name,Index Year,Overall Score,Property Rights,Government Integrity,Judicial Effectiveness,Tax Burden,Government Spending,Fiscal Health,Business Freedom,Labor Freedom,Monetary Freedom,Trade Freedom,Investment Freedom,Financial Freedom,Rank
0,Albania,2021,65.2,46.1,40.6,22.8,89.0,74.6,86.6,66.1,51.6,82.0,82.8,70.0,70.0,35.0
1,Armenia,2021,71.9,57.3,45.0,55.3,87.1,81.3,84.3,81.9,74.5,76.9,73.8,75.0,70.0,18.0
2,Austria,2021,73.9,86.8,84.8,83.5,45.7,29.1,90.0,72.6,68.4,81.7,84.0,90.0,70.0,13.0
3,Azerbaijan,2021,70.1,67.9,46.8,55.8,88.1,65.2,99.4,80.5,65.9,73.2,68.0,70.0,60.0,22.5
4,Belarus,2021,61.0,58.6,39.8,40.4,92.9,55.5,96.5,75.9,75.7,70.2,76.0,30.0,20.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,Sweden,1995,61.4,70.0,90.0,NaN,44.0,0.0,NaN,70.0,NaN,81.5,77.0,70.0,50.0,10.0
1221,Switzerland,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1222,Turkey,1995,58.4,70.0,10.0,NaN,44.5,63.3,NaN,85.0,NaN,38.2,74.8,70.0,70.0,14.0
1223,Ukraine,1995,39.9,30.0,10.0,NaN,61.8,47.1,NaN,55.0,NaN,0.0,55.0,50.0,50.0,23.0


In [104]:
ioef = ioef.sort_values(["Name", "Index Year"])
ioef["Rank Previous Year"] = ioef.groupby(["Name"])["Rank"].shift()
ioef

,Name,Index Year,Overall Score,Property Rights,Government Integrity,Judicial Effectiveness,Tax Burden,Government Spending,Fiscal Health,Business Freedom,Labor Freedom,Monetary Freedom,Trade Freedom,Investment Freedom,Financial Freedom,Rank,Rank Previous Year
1180,Albania,1995,49.7,50.0,10.0,NaN,81.7,34.3,NaN,70.0,NaN,22.1,59.0,70.0,50.0,20.0,NaN
1135,Albania,1996,53.8,50.0,10.0,NaN,81.7,51.0,NaN,70.0,NaN,42.6,59.0,70.0,50.0,27.0,20.0
1090,Albania,1997,54.8,50.0,10.0,NaN,81.5,60.3,NaN,70.0,NaN,62.9,59.0,70.0,30.0,30.0,27.0
1045,Albania,1998,53.9,30.0,10.0,NaN,82.1,66.5,NaN,70.0,NaN,67.7,59.0,70.0,30.0,31.0,30.0
1000,Albania,1999,53.4,30.0,10.0,NaN,82.9,72.5,NaN,70.0,NaN,57.8,57.8,70.0,30.0,33.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,United Kingdom,2017,76.4,93.8,78.3,93.0,65.1,41.9,40.4,89.9,72.8,85.0,87.0,90.0,80.0,4.0,4.0
183,United Kingdom,2018,78.0,92.2,79.0,93.8,65.2,44.4,53.5,91.1,74.4,85.2,86.9,90.0,80.0,4.0,4.0
137,United Kingdom,2019,78.9,92.3,83.8,85.9,64.7,48.2,68.6,92.9,73.5,81.2,86.0,90.0,80.0,3.0,4.0
91,United Kingdom,2020,79.3,92.2,89.9,82.7,64.7,49.5,78.1,94.7,73.1,80.3,86.4,80.0,80.0,3.0,3.0


In [105]:
ioef["Difference"] = ioef["Rank"] - ioef["Rank Previous Year"]
ioef.head()

,Name,Index Year,Overall Score,Property Rights,Government Integrity,Judicial Effectiveness,Tax Burden,Government Spending,Fiscal Health,Business Freedom,Labor Freedom,Monetary Freedom,Trade Freedom,Investment Freedom,Financial Freedom,Rank,Rank Previous Year,Difference
1180,Albania,1995,49.7,50.0,10.0,NaN,81.7,34.3,NaN,70.0,NaN,22.1,59.0,70.0,50.0,20.0,NaN,NaN
1135,Albania,1996,53.8,50.0,10.0,NaN,81.7,51.0,NaN,70.0,NaN,42.6,59.0,70.0,50.0,27.0,20.0,7.0
1090,Albania,1997,54.8,50.0,10.0,NaN,81.5,60.3,NaN,70.0,NaN,62.9,59.0,70.0,30.0,30.0,27.0,3.0
1045,Albania,1998,53.9,30.0,10.0,NaN,82.1,66.5,NaN,70.0,NaN,67.7,59.0,70.0,30.0,31.0,30.0,1.0
1000,Albania,1999,53.4,30.0,10.0,NaN,82.9,72.5,NaN,70.0,NaN,57.8,57.8,70.0,30.0,33.0,31.0,2.0


In [107]:
import sys
print(sys.version)
pandas.__version__

3.8.3 (tags/v3.8.3:6f8c832, May 13 2020, 22:20:19) [MSC v.1925 32 bit (Intel)]


'1.1.3'